In [27]:
import numpy as np
import warnings

from sklearn.feature_selection import VarianceThreshold,SelectKBest   # 删除所有低方差特征的特征选择器。 #   根据k个最高分选择功能。
from sklearn.feature_selection import f_regression   # 单变量线性回归测试。
from sklearn.feature_selection import chi2     #  计算每个非负特征和类之间的卡方统计量。
from sklearn.feature_selection import RFE      # 具有递归特征消除的特征排名。
from sklearn.feature_selection import SelectFromModel     # 元变换器，用于根据重要性权重选择特征。
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR   
from sklearn.ensemble import GradientBoostingClassifier
import  re
import os

In [29]:
test_words = '老师 会计师 程序员 律师 医生 老人'
re.split(r'\s+',test_words)
os.listdir('./')

['.ipynb_checkpoints',
 '01_案例一：TF-IDF.ipynb',
 '02_案例二：缺省值填充.ipynb',
 '03_案例三：哑编码.ipynb',
 '04_案例四：二值化.ipynb',
 '05_案例五：标准化.ipynb',
 '06_案例六：区间缩放法(归一化).ipynb',
 '07_案例七：正则化.ipynb',
 '08_案例八：多项式转换.ipynb',
 '09_案例九：特征选择&降维.ipynb',
 '10_案例十：异常数据处理.ipynb',
 '11_案例十一：车辆数据预处理.ipynb',
 'C0904.csv',
 'C0911.csv',
 'car.data',
 'datas',
 'nlp_test1.txt',
 'word2vec.mm',
 '分词案例1：jieba基础操作.ipynb',
 '分词案例2：jieba中文案例.ipynb',
 '扩展Word2Vec案例：gensim初识.ipynb']

In [17]:
X = np.array([
    [0, 2, 0, 3],
    [0, 1, 4, 3],
    [0.1, 1, 1, 3]
], dtype=np.float32)
Y = np.array([1,2,1])

In [18]:
variance = VarianceThreshold(threshold=0.1)
print(variance)
variance.fit(X)
print('-----------------')
print(variance.transform(X))

VarianceThreshold(threshold=0.1)
-----------------
[[ 2.  0.]
 [ 1.  4.]
 [ 1.  1.]]


In [19]:
sk1 = SelectKBest(f_regression, k=2)
sk1.fit(X, Y)
print(sk1)
print('------------')
print(sk1.scores_)
print('------------')
print(sk1.transform(X))

SelectKBest(k=2, score_func=<function f_regression at 0x000001BC4F2CD378>)
------------
[  0.33333333   0.33333333  16.33333333          nan]
------------
[[ 2.  0.]
 [ 1.  4.]
 [ 1.  1.]]


C:\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [20]:
# 使用chi2的时候要求特征属性为非负数
sk2 = SelectKBest(chi2, k=2)
sk2.fit(X, Y)
print(sk2)
print(sk2.scores_)
print(sk2.transform(X))

SelectKBest(k=2, score_func=<function chi2 at 0x000001BC4F2CD2F0>)
[ 0.05   0.125  4.9    0.   ]
[[ 2.  0.]
 [ 1.  4.]
 [ 1.  1.]]


In [21]:
estimator = SVR(kernel='linear')
selector = RFE(estimator, 2, step=1)
selector = selector.fit(X, Y)
print(selector.support_)
print(selector.n_features_)
print(selector.ranking_)
print(selector.transform(X))

[False  True  True False]
2
[2 1 1 3]
[[ 2.  0.]
 [ 1.  4.]
 [ 1.  1.]]


In [22]:
X2 = np.array([
    [ 5.1,  3.5,  1.4,  0.2],
    [ 4.9,  3. ,  1.4,  0.2],
    [ -6.2,  0.4,  5.4,  2.3],
    [ -5.9,  0. ,  5.1,  1.8]
], dtype=np.float64)
Y2 = np.array([0, 0, 2, 2])
estimator = LogisticRegression(penalty='l1', C=0.1)
sfm = SelectFromModel(estimator)
sfm.fit(X2, Y2)
print(sfm.transform(X2))

[[ 5.1]
 [ 4.9]
 [-6.2]
 [-5.9]]


In [23]:
X2 = np.array([
    [ 5.1,  3.5,  1.4,  0.2],
    [ 4.9,  3. ,  1.4,  0.2],
    [ -6.2,  0.4,  5.4,  2.3],
    [ -5.9,  0. ,  5.1,  1.8]
], dtype=np.float64)
Y2 = np.array([0, 0, 2, 2])
estimator = GradientBoostingClassifier()
sfm = SelectFromModel(estimator)
sfm.fit(X2, Y2)
print(sfm.transform(X2))

[[ 5.1  0.2]
 [ 4.9  0.2]
 [-6.2  2.3]
 [-5.9  1.8]]


In [32]:
from sklearn.decomposition import PCA
X2 = np.array([
    [ 5.1,  3.5,  1.4,  0.2, 1, 23],
    [ 4.9,  3. ,  1.4,  0.2, 2.3, 2.1],
    [ -6.2,  0.4,  5.4,  2.3, 2, 23],
    [ -5.9,  0. ,  5.1,  1.8, 2, 3]
], dtype=np.float64)
# n_components: 给定降低到多少维度，但是要求该值必须小于等于样本数目/特征数目，如果给定的值大于，那么会选择样本数目/特征数目中最小的那个作为最终的特征数目
pca = PCA(n_components=4)
pca.fit(X2)
print(pca.mean_)
print(pca.components_)
print(pca.transform(X2))

[-0.525  1.725  3.325  1.125  1.825 12.775]
[[ 0.02038178 -0.01698103 -0.01350052 -0.0149724   0.03184796 -0.99893718]
 [ 0.9024592   0.25030511 -0.31422084 -0.15092666 -0.03185873  0.01965141]
 [-0.08872116 -0.06952185 -0.06858116 -0.3074396  -0.94204108 -0.02512755]
 [ 0.38648983 -0.28075603  0.87190211  0.04987478 -0.09535412 -0.00291293]]
[[-1.01160631e+01  6.49232600e+00  3.14197238e-01 -4.61095452e-16]
 [ 1.08075405e+01  5.73455069e+00 -3.32785235e-01  1.10903256e-17]
 [-1.03473322e+01 -6.08709685e+00 -3.29724759e-01  4.20144074e-16]
 [ 9.65585479e+00 -6.13977984e+00  3.48312756e-01  7.94844344e-16]]


In [36]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = np.array([
    [-1, -1, 3, 1], 
    [-2, -1, 2, 4], 
    [-3, -2, 4, 5], 
    [1, 1, 5, 4], 
    [2, 1, 6, -5], 
    [3, 2, 1, 5]])
y = np.array([1, 1, 2, 2, 0, 1])
# n_components：给定降低到多少维度，要求给定的这个值和y的取值数量有关，不能超过n_class-1
clf = LinearDiscriminantAnalysis(n_components=2)
clf.fit(X, y)
print(clf.transform(X))

[[-3.2688434  -0.38911349]
 [-1.25507558 -1.78088569]
 [ 5.26064254 -0.49688862]
 [ 6.34385833  1.16134391]
 [-4.05800618  3.58297801]
 [-3.02257571 -2.07743411]]


d:\python\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
